# The Battle of Neighborhoods (Part I)

#####  By: Aminath Shausan

### Background

As we hit  halfway point in 2020,   this year appears challenging to the residents of Queensland, Australia. With the experience of fires and floods in the beginning of the year and currently the spread of Coronavirus (COVID-19), there are so many uncertainities in  the share market. Meanwhile the Government and  the Reserve Bank of Australia have quickly taken  steps to reduce interest rates and introduced huge stimulus packages to combat these issues, how is COVID-19 affecting the property market in Brisbane? 

According to research by Hunter Galloway (https://www.huntergalloway.com.au/), the property market in Brisbane has been seen a strong start at the beginning of 2020 and has a very strong housing affordability compared to Sydney and Melbourne. The current price to income ratio required to buy in Brisbane is 5.3 times at a median house price of $524,000. However, not all Brisbane suburbs are equally competitive for home buyers when it comes for long term investment. As per the above mentioned research, some suburbs such as Greenslope, Strathpine, Ferny Hills, Kedron, Oxley, Caloundra   are expected to shine in the property market due to their proximity to the Brisbane CBD. Whilst proximity to Brisbane CBD is a criterior for high demand for home buyers, those who have school aged children prefer to buy houses in a school catchman with a high rank.  Suprisingly, there are some suburbs such as Brisbane CBD, Milton, Kangaroo Point at a risk for investors as some banks may apply stricter criteria to loans, could reduce rental income and may ask for more deposit.

### The Problem

With the above observation that not every suburb in Brisbane is not equally competitive for investment, I aim to apply machine learning tools to assist people who want to invest in Brisbane housing market to make a sound decision.  The specific business problem which I intend to answer is: which suburbs in Brisbane are suitable for investment in this uncertain economic and pandamic situation?    

In order to solve this problem, I am going to cluster Brisbane suburbs, in the Greater Brisbane area, according to current real estate median  house prices and venues and essential facilities such as schools, cafes, hospitals and grocery stores in those suburbs.  This clustering will provide to determine which suburbs are best livable according to their median house prices and venues and thus can be recommended to investors.   
  

### Description of Data and its Application

Median house value data on the top 1408 Queensland suburbs are extracted from http://house.speakingsame.com/ website. This dataset consists of three columns; Rank, Suburb, Value. The 'Rank' column corresponds to the rank of the each suburb according to its median house price (corresponding to the 'Value' column) and the 'Suburb' column provides the name of the suburb. From this dataset, those rows corresponding to the Greater Brisbane suburbs are filtered. The names of the Greater Brisbane suburbs are collected from Wikipedia (https://en.wikipedia.org/wiki/List_of_Brisbane_suburbs).   

In the next step, Python's 'geopy' package is used to add location of each Brisbane suburbs.  Then FourSquare API is used to extract venues and essential facilities in each suburb and then this data is merged with the Brisbande house price dataset. Finaly, by performing a k-means clustering on this combined dataset, we will be able to recommend suitable and profitable suburbs to investors.   

### Methodology 

This section describes the data collection and preprocessing, the analysis  and prediction parts. It comprises of the following subsections 

1. Data collection and its Study:- webscrape house price data on the top 1408 Queensland suburbs from http://house.speakingsame.com/ and filter those corresponding to Greater Brisbane suburbs given in https://en.wikipedia.org/wiki/List_of_Brisbane_suburbs. Note that the data on all Queensland suburbs is contained in 28 webpages. View the first five rows and size of dataset, rename columns
2. Preprocessing data:- add suburb location data, produce a map containing suburbs and prices (may need to restrict data to specific budget if required), extract venue  and essential facilities information from FourSquare API, perform onehot encoding.
3. Modelling :- perform k-means clustering
4. Prediction:- determine top 10 or 5 clusters


#### 1.  Data Collection and its Study 

In this subsection, the median house price data for 1408 Queensland suburbs are collected by  webscraping 28 websites corresponding to  http://house.speakingsame.com/. I will first demonstrate how to collect this data from the first webpage (http://house.speakingsame.com/suburbtop.php?sta=qld&cat=HomePrice&name=&page=0) and filter those data corresponding to the Greater Brisbane suburbs, so that it is easier to learn the structure of the dataset. I will then use a 'for'  loop which runs through this process for all 28 webpages. 

Let me first import the required Python libraries for this process.

In [2]:
#import required libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported.')

Libraries imported.


We also need a list containing all suburbs in the Greater Brisbane suburbs. We form this list using the list of suburbs given in https://en.wikipedia.org/wiki/List_of_Brisbane_suburbs, which distinguishes Greater Brisbane suburbs into 5 categories; inner, northern, southern, eastern and western suburbs. There are altogether 192 suburbs in this list.

In [3]:
#create a list of  suburb names of Greater Brisbane
inner_sub = ['Bowen Hills',  'Brisbane', 'East Brisbane', 'Fortitude Valley', 'Herston', 'Highgate Hill', 'Kangaroo Point', 'Kelvin Grove', 
             'New Farm', 'Newstead', 'Paddington', 'Petrie Terrace', 'Red Hill', 'South Brisbane', 'Spring Hill', 'Teneriffe', 'West End', 
             'Woolloongabba']

north_sub = ['Albion', 'Alderley', 'Ascot', 'Aspley', 'Bald Hills', 'Banyo', 'Boondall', 'Bracken Ridge', 'Bridgeman Downs', 'Brighton',
             'Brisbane Airport', 'Carseldine', 'Chermside', 'Chermside West', 'Clayfield',  'Deagon', 'Eagle Farm', 'Everton Park', 'Ferny Grove',
             'Fitzgibbon', 'Gaythorne', 'Geebung', 'Gordon Park', 'Grange', 'Hamilton', 'Hendra', 'Kalinga', 'Kedron', 'Keperra', 'Lutwyche', 
             'McDowall', 'Mitchelton', 'Myrtletown', 'Newmarket', 'Northgate', 'Nudgee', 'Nudgee Beach', 'Nundah', 'Pinkenba', 'Sandgate',
             'Shorncliffe', 'Stafford', 'Stafford Heights', 'Taigum', 'Virginia', 'Wavell Heights', 'Wilston', 'Windsor', 'Wooloowin', 'Zillmere']

south_sub = ['Acacia Ridge', 'Algester', 'Annerley', 'Archerfield', 'Burbank', 'Calamvale', 'Coopers Plains', 'Darra', 'Doolandella', 
             'Drewvale', 'Durack', 'Dutton Park', 'Eight Mile Plains', 'Ellen Grove', 'Fairfield', 'Forest Lake', 'Greenslopes', 'Heathwood', 
             'Holland Park', 'Holland Park West', 'Inala', 'Karawatha', 'Kuraby', 'Larapinta', 'MacGregor', 'Mackenzie', 'Mansfield', 'Moorooka', 
             'Mount Gravatt', 'Mount Gravatt East', 'Nathan', 'Pallara', 'Parkinson', 'Richlands', 'Robertson', 'Rochedale', 'Rocklea', 
             'Runcorn', 'Salisbury', 'Seventeen Mile Rocks',  'Sinnamon Park', 'Stones Corner', 'Stretton', 'Sumner', 'Sunnybank',
             'Sunnybank Hills', 'Tarragindi',  'Tennyson', 'Upper Mount Gravatt', 'Wacol', 'Willawong',  'Wishart', 'Yeerongpilly', 'Yeronga']

east_sub = ['Balmoral', 'Belmont', 'Bulimba', 'Camp Hill', 'Cannon Hill', 'Carina', 'Carina Heights', 'Carindale', 'Chandler', 'Coorparoo', 
            'Gumdale', 'Hawthorne', 'Hemmant', 'Lota', 'Lytton', 'Manly', 'Manly West', 'Moreton Island', 'Morningside', 'Murarrie', 
            'Norman Park', 'Port of Brisbane', 'Ransome', 'Seven Hills', 'Tingalpa', 'Wakerley', 'Wynnum', 'Wynnum West']

west_sub = ['Anstead', 'Ashgrove', 'Auchenflower', 'Bardon', 'Bellbowrie', 'Brookfield', 'Chapel Hill', 'Chelmer', 'Chuwar', 'Corinda', 'Enoggera', 
            'Enoggera Reservoir', 'Fig Tree Pocket', 'Graceville', 'Indooroopilly', 'Jamboree Heights' , 'Jindalee', 'Karana Downs', 
            'Kenmore', 'Kenmore Hills', 'Kholo', 'Lake Manchester', 'Middle Park', 'Milton', 'Moggill', 'Mount Coot-tha', 'Mount Crosby', 
            'Mount Ommaney', 'Oxley', 'Pinjarra Hills', 'Pullenvale', 'Riverhills', 'Seventeen Mile Rocks', 'Sherwood', 'Sinnamon Park', 
            'St Lucia', 'Taringa', 'The Gap', 'Toowong', 'Upper Brookfield', 'Upper Kedron', 'Westlake']

#check length of the list 
#print(len(inner_sub), len(north_sub), len(south_sub), len(east_sub), len(west_sub))

#join the list of suburbs into one list 
sub_list  = inner_sub + north_sub + south_sub + east_sub + west_sub
print(len(sub_list))

192


Now lets read data on median house prices from the first webpage (http://house.speakingsame.com/suburbtop.php?sta=qld&cat=HomePrice&name=&page=0) and inspect its size and the first 5 rows.

In [4]:
#read the table in the http://house.speakingsame.com/suburbtop.php?sta=qld&cat=HomePrice&name=&page=0

df = pd.read_html('http://house.speakingsame.com/suburbtop.php?sta=qld&cat=HomePrice&name=&page=0')[5]

#check size and 1st few rows
print(df.shape)
df.head(5)

(51, 3)


,0,1,2
0,Rank,Suburb,Value
1,1,Jollys Lookout,"$2,999,900"
2,2,Sovereign Islands,"$2,619,400"
3,3,Mermaid Beach,"$2,200,100"
4,4,Sunshine Beach,"$1,975,100"


There are 51 rows and 3 columns in this dataframe. Notice that row 0 actually contains the column names in the actual dataset. So we will rename the columns with these names and drop row 0. This implies that the actual dataframe will consist of 50 rows and 3 columns

In [5]:
#rename columns as 'Rank', 'Suburb', 'Value' and drop the first row
df.columns = ['Rank', 'Suburb', 'Value']
df.drop(df.index[0], inplace=True)
  
#check size and 1st few rows
print(df.shape)
df.head()    

(50, 3)


,Rank,Suburb,Value
1,1,Jollys Lookout,"$2,999,900"
2,2,Sovereign Islands,"$2,619,400"
3,3,Mermaid Beach,"$2,200,100"
4,4,Sunshine Beach,"$1,975,100"
5,5,Cooroy Mountain,"$1,946,600"


In [61]:
#check which suburbs in df 'Suburb' columns are also in sub_list 

df_sub = df['Suburb'].tolist()
#set(df_sub).intersection(sub_list)
com_sub = set(df_sub) & set(sub_list)
print(len(com_sub))    #There   are 22 common suburbs names in both lists
print(len(df_sub))

22
50


Lets filter, from df dataframe, those suburbs containing in the list sub_list

In [6]:
#filter suburbs in df which corresponds to those in sub_list
df1 = df.query('Suburb in @sub_list')
#df1 = df.loc[(df['Suburb'].isin(sub_list))] 

#check size and 1st few rows
print(df1.shape)
df1.head()   


(22, 3)


,Rank,Suburb,Value
10,10,Willawong,"$1,552,100"
11,11,New Farm,"$1,443,500"
15,15,Teneriffe,"$1,401,200"
16,16,Gumdale,"$1,367,000"
18,18,Hamilton,"$1,357,700"


Now, I will write a 'for' loop which does the above process for the 28 webpages containing required data. The urls for each webpage differs by the last number, with the first webpage url ending with 0 (http://house.speakingsame.com/suburbtop.php?sta=qld&cat=HomePrice&name=&page=0), and incrementing by 1 for the next webpage, and the last webpage ending with number 28.   

In [7]:
#webscrap all 28 webpages, filter data, according to suburb names in sub_list
urls = {page_number:'http://house.speakingsame.com/suburbtop.php?sta=qld&cat=HomePrice&name=&page={}'.format(page_number) 
            for page_number in list(range(0, 29))}


dfs ={}       # a dictionary to hold the original (unfiltered) dataframes
dfnew = {}    # a dictionary to hold the filtered dataframes

for url in range(0,len(urls)):
    dfs[url]= (pd.read_html(urls[url])[5])             # read data from webpage
    dfs[url].columns = ['Rank', 'Suburb', 'Value']     # rename columns
    dfs[url].drop(dfs[url].index[0], inplace=True)     # drop the first row
    dfnew[url] = dfs[url].query('Suburb in @sub_list') # filter according to suburb names in the sub_list    


Check if the size and first 5 rows of dfnew[0] matches with that of df1, whose size is 22 rows and 3 columns

In [8]:
#check if size and 1st 5 rows of dfnew[1] matches with that of df1 above
print(dfnew[0].shape)
dfnew[0].head()  


(22, 3)


,Rank,Suburb,Value
10,10,Willawong,"$1,552,100"
11,11,New Farm,"$1,443,500"
15,15,Teneriffe,"$1,401,200"
16,16,Gumdale,"$1,367,000"
18,18,Hamilton,"$1,357,700"


Since the size and first 5 rows of df1 and dfnew[0] are the same, we can safely assume that the for loop is correctly extracting data corresponding to those suburbs in sub_list list. Thus we can now combine all dataframes in the dictionary dfnew into a single dataframe.  

In [9]:
#merge all dataframes in dfnew dictionary vertically 

brisbane_df = pd.concat(dfnew.values(), ignore_index=True)
print(brisbane_df.shape)
brisbane_df.head()

(178, 3)


,Rank,Suburb,Value
0,10,Willawong,"$1,552,100"
1,11,New Farm,"$1,443,500"
2,15,Teneriffe,"$1,401,200"
3,16,Gumdale,"$1,367,000"
4,18,Hamilton,"$1,357,700"


Note that the dataframe brisbane_df contains data on 178 suburbs which means that not all suburb median prices in the Greater Brisbane region are given in the webpages  http://house.speakingsame.com/. Thus for illustration purpose and considering time to manually enter house prices for those suburbs not included in the sub_list list, we will analyse data only for those  suburbs in the brisbane_df. 

In [31]:
#urls = {page_number:'http://house.speakingsame.com/suburbtop.php?sta=qld&cat=HomePrice&name=&page={}'.format(page_number) for page_number in list(range(0,2))}

#print("Get urls")

#for url in range(0,len(urls)):
#    print(urls[url], end=', ')

#print(dfs.keys())
#print(len(dfs.keys()))
#df_new = dfs[28]
#print(df_new.shape)
#len(urls)
#print(list(range(0,len(urls))))

#df_new.head()

In [28]:
#filter the rows corresponding to Brisbane suburbs

#dfnew = {}
#for df in range(0,len(dfs.keys())):
#    dfnew[df] = dfs[df].loc[(dfs[df]['Suburb'].isin(sub_list))] 
   

In [29]:
#print(dfnew[5].shape)
#dfnew[0].head()

In [32]:
##merge the dataframes vertically 
##brisbane_df = pd.concat([dfnew[0], dfnew[1]], ignore_index=True)

#brisbane_df = pd.concat(dfnew.values(), ignore_index=True)
#print(brisbane_df.shape)
#brisbane_df.head()

The combined df has 178 rows,  which does not match with the length of sub_list of 192. So we need to findout which suburbs are not in the brisbane_df and mannually add their values 

In [30]:
#check if all inner_sub are in the brisbane_df
#select_values = [31, 22, 30, 25, 64]
#df_inner = brisbane_df.query('Suburb in @inner_sub')
#df_north = brisbane_df.query('Suburb in @north_sub')

#print(df_inner.shape, len(inner_sub))
#print(df_north.shape, len(north_sub))
#df['one'].tolist()
#df_inner_sub = df_inner['Suburb'].tolist()
#df_north_sub = df_north['Suburb'].tolist()

#list(set(inner_sub)-set(df_inner_sub))
#list(set(north_sub)-set(df_north_sub))

#all these give common elements in the two lists
#set(df_inner_sub) & set(inner_sub)  
#set(df_inner_sub).intersection(inner_sub)
#bool(set(df_inner_sub).intersection(inner_sub))
#df_inner_sub


In [13]:
#from df1, extract only the specific suburbs 
inner_df =   df[df['Suburb'].isin(inner_sub)]
north_df =   df[df['Suburb'].isin(north_sub)]
south_df =   df[df['Suburb'].isin(south_sub)]
east_df =   df[df['Suburb'].isin(east_sub)]
west_df =   df[df['Suburb'].isin(west_sub)]

 
print(inner_df.shape,north_df.shape, south_df.shape, east_df.shape, west_df.shape)
#print(north_df.shape)
#print(inner_df)
#print(north_df)
west_df

(5, 3) (4, 3) (3, 3) (5, 3) (5, 3)


,Rank,Suburb,Value
19,19,Fig Tree Pocket,"$1,337,700"
21,21,Chelmer,"$1,318,800"
24,24,St Lucia,"$1,289,300"
35,35,Pullenvale,"$1,216,500"
50,50,Toowong,"$1,112,500"


In [33]:
#join_df =  df.loc[(df['Suburb'].isin(inner_sub))+(df['Suburb'].isin(north_sub))+(df['Suburb'].isin(south_sub))
#                   +(df['Suburb'].isin(east_sub))+ (df['Suburb'].isin(west_sub))]
#join_df = df[df['Suburb'].str.contains(sub_list, case=False)]

#join_df = df.loc[(df['Suburb'].isin(sub_list))]
#print(join_df.shape)
#join_df
